In [1]:
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%matplotlib inline

## NN on weather radar image

In this exercise I will develop my own NN and use it against the same weather radar image. 

- Gjer meg opp ein meining for kostfunksjon. ABS eller MSE. 
- gjør NN klart for ulike aktiveringsfunksjoner
- gjer det sammenlignbart med OLS og Ridge i forrige oppgave og prosjekt 1. 
- gjer det sammenlignbart med tensorflow/keras. 
- gjer det mulig å initialisere med ulike vekter og bias. 

## Different activation functions

- Test ut sigmoid, RELU og Leaky RELU.
- Mulig: Sjekk ut om måten eg initialiserer vekter og bias på på har noko å sei. 